In [ ]:
# Google Colab compatibility
project_folder = ''

In [ ]:
from custompackage.load_data import *
from custompackage.load_architecture import *
from custompackage.traintestloop import *
import numpy as np

In [ ]:
class_set = 0

# Initialize settings
bs = 256
weighting = 'paired'
trials = 10
epochs = 2000
trees_set = [1,32]

# Load class-dataset list
classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

# Initialize final test loss and accuracy variables
loss = np.zeros((len(classes), trials, len(trees_set)))
acc = np.zeros((len(classes), trials, len(trees_set)))


# For each dataset enumerated from classes list
for j, (t1, t2, ds) in enumerate(classes):
    print(t1, t2, ds)
    # Load data loaders
    trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds, permute=False)
    # Initialize input size for model initialization purposes
    input_size = trainloaders[0].dataset.tensors[0][0].shape[0]
    # For each trial
    for k, trees in enumerate(trees_set):

        # For every k-tree defined by trees_set
        for i in range(trials):
            print(j, i, k)
            # Initialize the fcnn model, such that hidden layer is twice the number of trees
            model = simple_fcnn(input_size, 2*trees, 1).cuda()
            #Train and test fcnn, assigning loss and acc values
            loss_curve, acc_curve, loss[j,i,k], acc[j,i,k], model_t = train_test_fc(model, trainloaders[i],
                                              validloaders[i], testloader, epochs=epochs)

            # Save accuracy and loss arrays
            np.save(project_folder + 'results/fcnn_acc_'+str(class_set)+'.npy', acc)
            np.save(project_folder + 'results/fcnn_loss_'+str(class_set)+'.npy', loss)

In [ ]:
class_set = 0
acc = np.load(project_folder + 'results/fcnn_acc_'+str(class_set)+'.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{k}-FCNN")
        print(f"Accuracy: mean = {round(np.mean(acc[j,:,k]), 4)}, standard deviation = {round(np.std(acc[j,:,k]), 4)}")